In [2]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
import pandas as pd
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf

In [35]:
def readData(filepath, label):
    cells = []
    labels = []
    file = os.listdir(filepath)
    for img in file:
        try:
            image = cv2.imread(filepath + img)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((50, 50))
            cells.append(np.array(size_image))
            labels.append(label)
        except AttributeError as e:
            print('Skipping file: ', img, e)
    print(len(cells), ' Data Points Read!')
    return np.array(cells), np.array(labels)

In [36]:
def genesis_train(file):
    
    print('Reading Training Data')
    
    ParasitizedCells, ParasitizedLabels = readData(file + '/Parasitized/', 1)
    UninfectedCells, UninfectedLabels  = readData(file + '/Uninfected/', 0)
    Cells = np.concatenate((ParasitizedCells, UninfectedCells))
    Labels = np.concatenate((ParasitizedLabels, UninfectedLabels))
    
    print('Reading Testing Data')
    
    TestParasitizedCells, TestParasitizedLabels = readData('./input/fed/test/Parasitized/', 1)
    TestUninfectedCells, TestUninfectedLabels  = readData('./input/fed/test/Uninfected/', 0)
    TestCells = np.concatenate((TestParasitizedCells, TestUninfectedCells))
    TestLabels = np.concatenate((TestParasitizedLabels, TestUninfectedLabels))
    
    s = np.arange(Cells.shape[0])
    np.random.shuffle(s)
    Cells = Cells[s]
    Labels = Labels[s]
    
    sTest = np.arange(TestCells.shape[0])
    np.random.shuffle(sTest)
    TestCells = TestCells[sTest]
    TestLabels = TestLabels[sTest]
    
    num_classes=len(np.unique(Labels))
    len_data=len(Cells)
    print(len_data, ' Data Points')
    
    (x_train,x_test)=Cells, TestCells
    (y_train,y_test)=Labels, TestLabels
    
    # Since we're working on image data, we normalize data by divinding 255.
    x_train = x_train.astype('float32')/255 
    x_test = x_test.astype('float32')/255
    train_len=len(x_train)
    test_len=len(x_test)
    
    #Doing One hot encoding as classifier has multiple classes
    y_train=keras.utils.to_categorical(y_train,num_classes)
    y_test=keras.utils.to_categorical(y_test,num_classes)
    
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
#     model.summary()

    # compile the model with loss as categorical_crossentropy and using adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
    model.fit(x_train, y_train, batch_size=100, epochs=5, verbose=1)
    
    scores = model.evaluate(x_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    model.save("./weights/global1.h5")
    return len_data, scores[0], scores[1]

In [37]:
def local_train(file, d, globalId):
    
    print('Reading Training Data')
    
    ParasitizedCells, ParasitizedLabels = readData(file + '/Parasitized/', 1)
    UninfectedCells, UninfectedLabels  = readData(file + '/Uninfected/', 0)
    Cells = np.concatenate((ParasitizedCells, UninfectedCells))
    Labels = np.concatenate((ParasitizedLabels, UninfectedLabels))
    
    print('Reading Testing Data')
    
    TestParasitizedCells, TestParasitizedLabels = readData('./input/fed/test/Parasitized/', 1)
    TestUninfectedCells, TestUninfectedLabels  = readData('./input/fed/test/Uninfected/', 0)
    TestCells = np.concatenate((TestParasitizedCells, TestUninfectedCells))
    TestLabels = np.concatenate((TestParasitizedLabels, TestUninfectedLabels))
    
    s = np.arange(Cells.shape[0])
    np.random.shuffle(s)
    Cells = Cells[s]
    Labels = Labels[s]
    
    sTest = np.arange(TestCells.shape[0])
    np.random.shuffle(sTest)
    TestCells = TestCells[sTest]
    TestLabels = TestLabels[sTest]
    
    num_classes=len(np.unique(Labels))
    len_data=len(Cells)
    print(len_data, ' Data Points')
    
    (x_train,x_test)=Cells, TestCells
    (y_train,y_test)=Labels, TestLabels
    
    # Since we're working on image data, we normalize data by divinding 255.
    x_train = x_train.astype('float32')/255 
    x_test = x_test.astype('float32')/255
    train_len=len(x_train)
    test_len=len(x_test)
    
    #Doing One hot encoding as classifier has multiple classes
    y_train=keras.utils.to_categorical(y_train,num_classes)
    y_test=keras.utils.to_categorical(y_test,num_classes)
    
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()

    model.load_weights("./weights/global"+str(globalId)+".h5")
    
    # compile the model with loss as categorical_crossentropy and using adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
    model.fit(x_train, y_train, batch_size=100, epochs=5, verbose=1)
    
    
    scores = model.evaluate(x_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    model.save("./weights/" + str(d) + ".h5")
    return len_data, scores[0], scores[1]

In [38]:
#### Zeno ####

def getDataLen(trainingDict):
    n = 0
    for w in trainingDict:
        n += trainingDict[w][0]
    print('Total number of data points after this round: ', n)
    return n

def assignWeights(trainingDf, trainingDict):
    n = getDataLen(trainingDict)
    trainingDf['Weightage'] = trainingDf['DataSize'].apply(lambda x: x/n)
    return trainingDf, n
    
def scale(weight, scaler):
    scaledWeights = []
    for i in range(len(weight)):
        scaledWeights.append(scaler * weight[i])
    return scaledWeights

def getScaledWeight(d, scaler):
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return scale(weight, scaler)

def avgWeights(scaledWeights):
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(trainingDict):
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize', 'Loss','Accuracy']) 
    models = []
    for i in trainingDict.keys():
        models.append(i)
    scaledWeights = []
    trainingDf, dataLen = assignWeights(trainingDf, trainingDict)
    for m in models:
        scaledWeights.append(getScaledWeight(m, trainingDf.loc[m]['Weightage']))
    fedAvgWeight = avgWeights(scaledWeights)
    return fedAvgWeight, dataLen


def ZenoScore(local, g_loss, p):
    scored = {}
    for l in local.keys():
        score = g_loss - local[l][1] - (local[l][0]*p)
        scored[l] = score
    return scored

def zeno(trainingDict, p, b):
    
    models = list(trainingDict.keys())
    
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize', 'Loss', 'Accuracy']) 
    
    g_loss = 0    
    local = {}
    
    for i in trainingDict.keys():
        if 'global' in i:
            g_loss = trainingDict[i][1]
        else:
            local[i] = (trainingDict[i])
            
    
    scores = ZenoScore(local, g_loss, p)
    b = int(len(scores)*b)
    sortedScores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1])}
    
    zenoedModels = []
    
    for i in range(b):
        zenoedModels.append((sortedScores.popitem())[0])
        
    newDict = {}
    for i in trainingDict.keys():
        if i not in zenoedModels:
            newDict[i] = trainingDict[i]
            
    print('Zeno Selections: ', zenoedModels)
        
#     NewGlobal, dataLen = FedAvg(trainingDict) !!!!!
    NewGlobal, dataLen = FedAvg(zenoedModels)
    
    return NewGlobal, dataLen

def saveModel(weight, n):
    
    print('Reading Testing Data')
    
    TestParasitizedCells, TestParasitizedLabels = readData('./input/fed/test/Parasitized/', 1)
    TestUninfectedCells, TestUninfectedLabels  = readData('./input/fed/test/Uninfected/', 0)
    TestCells = np.concatenate((TestParasitizedCells, TestUninfectedCells))
    TestLabels = np.concatenate((TestParasitizedLabels, TestUninfectedLabels))
    
    
    sTest = np.arange(TestCells.shape[0])
    np.random.shuffle(sTest)
    TestCells = TestCells[sTest]
    TestLabels = TestLabels[sTest]
    
    num_classes=len(np.unique(TestLabels))
    
    (x_test) = TestCells
    (y_test) = TestLabels
    
    # Since we're working on image data, we normalize data by divinding 255.
    x_test = x_test.astype('float32')/255
    test_len=len(x_test)
    
    #Doing One hot encoding as classifier has multiple classes
    y_test=keras.utils.to_categorical(y_test,num_classes)

    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    model.set_weights(weight)

    # compile the model with loss as categorical_crossentropy and using adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    scores = model.evaluate(x_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    fpath = "./weights/global"+n+".h5"
    model.save(fpath)
    return scores[0], scores[1]

In [39]:
globalDict = dict()
trainingDict = dict()
trainingDict['global1'] = genesis_train('./input/fed/genesis')
globalDict['global1'] = trainingDict['global1']

Reading Training Data
686  Data Points Read!
696  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1382  Data Points
Epoch 1/5
14/14 [==============================] - 2s 134ms/step - loss: 0.7411 - accuracy: 0.4906
Epoch 2/5
14/14 [==============================] - 2s 121ms/step - loss: 0.6945 - accuracy: 0.5123
Epoch 3/5
14/14 [==============================] - 2s 117ms/step - loss: 0.6868 - accuracy: 0.5839
Epoch 4/5
14/14 [==============================] - 2s 120ms/step - loss: 0.6703 - accuracy: 0.6085
Epoch 5/5
173/173 [==============================] - 2s 13ms/step - loss: 0.6063 - accuracy: 0.6701
Loss:  0.6062763929367065
Accuracy:  0.670106828212738


In [40]:
trainingDict['d1'] = local_train('./input/fed/d1', 'd1', 1)
trainingDict['d2'] = local_train('./input/fed/d2', 'd2', 1)
trainingDict['d3'] = local_train('./input/fed/d3', 'd3', 1)
trainingDict['d4'] = local_train('./input/fed/d4', 'd4', 1)
trainingDict['d5'] = local_train('./input/fed/d5', 'd5', 1)

Reading Training Data
528  Data Points Read!
533  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1061  Data Points
Epoch 1/5
11/11 [==============================] - 1s 110ms/step - loss: 0.6048 - accuracy: 0.6598
Epoch 2/5
11/11 [==============================] - 1s 103ms/step - loss: 0.5813 - accuracy: 0.7022
Epoch 3/5
11/11 [==============================] - 1s 112ms/step - loss: 0.5504 - accuracy: 0.7418
Epoch 4/5
11/11 [==============================] - 1s 126ms/step - loss: 0.5368 - accuracy: 0.7276
Epoch 5/5
173/173 [==============================] - 3s 15ms/step - loss: 0.5252 - accuracy: 0.7509
Loss:  0.5252317786216736
Accuracy:  0.7508600354194641
Reading Training Data
522  Data Points Read!
528  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1050  Data Points
Epoch 1/5
11/11 [==============================] - 1s 85ms/step - loss: 0.6164 - accuracy: 0.6571
Epoch 2/5
11/11 [=======================

In [41]:
trainingDict

{'global1': (1382, 0.6062763929367065, 0.670106828212738),
 'd1': (1061, 0.5252317786216736, 0.7508600354194641),
 'd2': (1050, 0.5273655652999878, 0.7318486571311951),
 'd3': (1347, 0.4978816509246826, 0.761904776096344),
 'd4': (858, 0.5582518577575684, 0.7054136991500854),
 'd5': (1676, 0.47650906443595886, 0.7984790802001953)}

In [42]:
NewGlobal, dataLen = zeno(trainingDict, 0.0005, 0.75)
trainingDict = {}
newModelLoss, newModelAccuracy = saveModel(NewGlobal, '2')
trainingDict['global2'] = (dataLen, newModelLoss, newModelAccuracy)

Zeno Selections:  ['d4', 'd2', 'd1']
Total number of data points after this round:  7374
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 9ms/step - loss: 0.5277 - accuracy: 0.7358
Loss:  0.5276505947113037
Accuracy:  0.7358319759368896


In [43]:
globalDict['global2'] = trainingDict['global2']
trainingDict['d6'] = local_train('./input/fed/d6', 'd6', 2)
trainingDict['d7'] = local_train('./input/fed/d7', 'd7', 2)
trainingDict['d8'] = local_train('./input/fed/d8', 'd8', 2)
trainingDict['d9'] = local_train('./input/fed/d9', 'd9', 2)
trainingDict['d10'] = local_train('./input/fed/d10', 'd10', 2)

Reading Training Data
599  Data Points Read!
567  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1166  Data Points
Epoch 1/5
12/12 [==============================] - 1s 95ms/step - loss: 0.5461 - accuracy: 0.7247
Epoch 2/5
12/12 [==============================] - 1s 101ms/step - loss: 0.4951 - accuracy: 0.7710
Epoch 3/5
12/12 [==============================] - 1s 91ms/step - loss: 0.4515 - accuracy: 0.8036
Epoch 4/5
12/12 [==============================] - 1s 91ms/step - loss: 0.3985 - accuracy: 0.8259
Epoch 5/5
173/173 [==============================] - 2s 10ms/step - loss: 0.3458 - accuracy: 0.8678
Loss:  0.3457932472229004
Accuracy:  0.867825448513031
Reading Training Data
418  Data Points Read!
395  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
813  Data Points
Epoch 1/5
9/9 [==============================] - 1s 83ms/step - loss: 0.5748 - accuracy: 0.7060
Epoch 2/5
9/9 [==============================] 

In [44]:
NewGlobal, dataLen = zeno(trainingDict, 0.0005, 0.75)
trainingDict = {}
newModelLoss, newModelAccuracy = saveModel(NewGlobal, '3')
trainingDict['global3'] = (dataLen, newModelLoss, newModelAccuracy)

Zeno Selections:  ['d7', 'd6', 'd9']
Total number of data points after this round:  13296
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 9ms/step - loss: 0.4655 - accuracy: 0.7773
Loss:  0.4655492901802063
Accuracy:  0.777294933795929


In [45]:
globalDict['global3'] = trainingDict['global3']
trainingDict['d11'] = local_train('./input/fed/d11', 'd11', 3)
trainingDict['d12'] = local_train('./input/fed/d12', 'd12', 3)
trainingDict['d13'] = local_train('./input/fed/d13', 'd13', 3)
trainingDict['d14'] = local_train('./input/fed/d14', 'd14', 3)
trainingDict['d15'] = local_train('./input/fed/d15', 'd15', 3)

Reading Training Data
557  Data Points Read!
577  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1134  Data Points
Epoch 1/5
12/12 [==============================] - 1s 90ms/step - loss: 0.5010 - accuracy: 0.7584
Epoch 2/5
12/12 [==============================] - 1s 88ms/step - loss: 0.4358 - accuracy: 0.8113
Epoch 3/5
12/12 [==============================] - 1s 91ms/step - loss: 0.3958 - accuracy: 0.8236
Epoch 4/5
12/12 [==============================] - 1s 90ms/step - loss: 0.3505 - accuracy: 0.8607
Epoch 5/5
173/173 [==============================] - 2s 10ms/step - loss: 0.3241 - accuracy: 0.8624
Loss:  0.3241368532180786
Accuracy:  0.8623936176300049
Reading Training Data
827  Data Points Read!
796  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
1623  Data Points
Epoch 1/5
17/17 [==============================] - 2s 93ms/step - loss: 0.4299 - accuracy: 0.8115
Epoch 2/5
17/17 [===========================

In [46]:
NewGlobal, dataLen = zeno(trainingDict, 0.0005, 0.75)
trainingDict = {}
newModelLoss, newModelAccuracy = saveModel(NewGlobal, '4')
trainingDict['global4'] = (dataLen, newModelLoss, newModelAccuracy)

Zeno Selections:  ['d15', 'd13', 'd14']
Total number of data points after this round:  18479
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 10ms/step - loss: 0.4200 - accuracy: 0.8050
Loss:  0.4200378656387329
Accuracy:  0.8049972653388977


In [47]:
globalDict['global4'] = trainingDict['global4']
trainingDict['d16'] = local_train('./input/fed/d16', 'd16', 4)
trainingDict['d17'] = local_train('./input/fed/d17', 'd17', 4)
trainingDict['d18'] = local_train('./input/fed/d18', 'd18', 4)
trainingDict['d19'] = local_train('./input/fed/d19', 'd19', 4)
trainingDict['d20'] = local_train('./input/fed/d20', 'd20', 4)

Reading Training Data
412  Data Points Read!
416  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
828  Data Points
Epoch 1/5
9/9 [==============================] - 1s 79ms/step - loss: 0.4813 - accuracy: 0.7838
Epoch 2/5
9/9 [==============================] - 1s 80ms/step - loss: 0.4078 - accuracy: 0.8345
Epoch 3/5
9/9 [==============================] - 1s 79ms/step - loss: 0.3486 - accuracy: 0.8623
Epoch 4/5
9/9 [==============================] - 1s 79ms/step - loss: 0.2994 - accuracy: 0.8816
Epoch 5/5
173/173 [==============================] - 2s 9ms/step - loss: 0.2890 - accuracy: 0.8847
Loss:  0.28904205560684204
Accuracy:  0.8846641182899475
Reading Training Data
417  Data Points Read!
414  Data Points Read!
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
831  Data Points
Epoch 1/5
9/9 [==============================] - 1s 84ms/step - loss: 0.4406 - accuracy: 0.8075
Epoch 2/5
9/9 [==============================] - 1s 85ms

In [48]:
NewGlobal, dataLen = zeno(trainingDict, 0.0005, 0.75)
trainingDict = {}
newModelLoss, newModelAccuracy = saveModel(NewGlobal, '5')
trainingDict['global5'] = (dataLen, newModelLoss, newModelAccuracy)
globalDict['global5'] = trainingDict['global5']

Zeno Selections:  ['d20', 'd18', 'd19']
Total number of data points after this round:  22035
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 9ms/step - loss: 0.4009 - accuracy: 0.8142
Loss:  0.400885671377182
Accuracy:  0.8142313957214355


In [49]:
globalDict

#Zeno

{'global1': (1382, 0.6062763929367065, 0.670106828212738),
 'global2': (7374, 0.5276505947113037, 0.7358319759368896),
 'global3': (13296, 0.4655492901802063, 0.777294933795929),
 'global4': (18479, 0.4200378656387329, 0.8049972653388977),
 'global5': (22035, 0.400885671377182, 0.8142313957214355)}